In [92]:
##https://github.com/LiorVanounou/MATALA.git
##ליאור ונונו
##313333106




In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif, RFE, RFECV
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import mean_squared_error, accuracy_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import datetime as dt 
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
def prepare_data(df):
    # מחיקת רווחים והסרת ירודים משמות העמודות
    df.columns = df.columns.str.strip().str.replace('\r\n', '').str.replace('\n', '').str.replace('\r', '')
    
    # הסרת עמודות לפי שם
    df.drop(columns=['Area', 'Cre_date', 'Repub_date', 'Supply_score', 'Test'], inplace=True)
    
    # נקיון עמודת MANUFACTURE
    df.manufactor = df.manufactor.str.replace('Lexsus', 'לקסוס')
    
    # נקיון עמודת MODEL
    words_to_remove = [
        "\r\n", "החדש", "הדור", "חדשה", 'אאודי', 'סקודה', 'ניסאן', 'לקסוס', 'סוזוקי', 'מאזדה', 'החדשה', r'\(\d+\)', 
        'רנו', 'אלפא רומיאו', 'יונדאי', 'ניסאן', 'סוזוקי', 'טויוטה', 'קיה', 'אאודי', 'סובארו', 'מיצובישי', 'מרצדס', 
        'ב.מ.וו', 'אופל', 'הונדה', 'פולקסווגן', 'שברולט', 'מאזדה', 'וולוו', 'סקודה', 'פורד', 'קרייזלר', 'סיטרואן', 
        "פיג'ו", 'רנו', 'לקסוס', 'דייהטסו', 'מיני', 'אלפא רומיאו'
    ]
    pattern = '|'.join(words_to_remove)
    df.model = df.model.str.replace(pattern, '', regex=True)
    df.model = df.model.str.strip()
    
    # פונקציה להחלפת ערך חסר בעמודת סוג הגיר
    def replace_gear(row, df):
        if pd.isna(row['Gear']) or row['Gear'] == 'לא מוגדר':
            mask = (df['model'] == row['model']) & (df['Year'] == row['Year']) & (df['manufactor'] == row['manufactor'])
            matching_rows = df[mask]
            if len(matching_rows) > 0:
                new_gear_value = matching_rows.iloc[0]['Gear']
                return new_gear_value
        return row['Gear']

    df['Gear'] = df.apply(replace_gear, axis=1, df=df)
    
    # החלפת ערכים לסוג מנוע
    df['Gear'] = df['Gear'].replace({'אוטומט': 'אוטומטית'})
    
    # החלפת ערכים לעמודת capacity_Engine
    replace_dict1 = {
        '132': '1320',
        '105': '1050',
        '110': '1100',
        '150': '1500',
        '13': '1300',
        '90': '900',
        '125': '1250',
        '80': '800',
        '12000': '1200'
    }
    df['capacity_Engine'] = df['capacity_Engine'].replace(replace_dict1, regex=True)
    
    # פונקציה להחלפת ערכים לא תקינים בנפח מנוע
    def fix_capacity_engine(df):
        for index, row in df.iterrows():
            capacity = row['capacity_Engine']
            if pd.notna(capacity) and isinstance(capacity, (int, float)) and len(str(capacity)) < 4:
                mask = (df['Gear'] == row['Gear']) & (df['Year'] == row['Year']) & (df['model'] == row['model'])
                matching_capacity = df.loc[mask, 'capacity_Engine'].values
                if len(matching_capacity) > 0:
                    df.at[index, 'capacity_Engine'] = matching_capacity[0]
        return df

    df = fix_capacity_engine(df)
    df.model = df.model.str.replace(pattern, '', regex=True)
    # החלפת ערכים לעמודת Engine_type
    df['capacity_Engine'] = df['capacity_Engine'].replace(",", '', regex=True)
    df['capacity_Engine'] = pd.to_numeric(df['capacity_Engine'], errors='coerce').fillna(0)

    # החלפת ערכים לעמודות Prev_ownership ו-Curr_ownership
    df['Prev_ownership'] = df['Prev_ownership'].replace('לא מוגדר', 'אחר', regex=True)
    df['Curr_ownership'] = df['Curr_ownership'].replace('לא מוגדר', 'אחר', regex=True)
    
    # החלפת ערכים לעמודת City
    replace_dict2 = {
        'jeruslem': 'ירושלים',
        'Rehovot': 'רחובות',
        'haifa': 'חיפה',
        'Rishon LeTsiyon': 'ראשון לציון',
        'פתח תקווה': 'פ"ת',
        'ashdod': 'אשדוד',
        'Tel aviv': 'תל אביב',
        'ראשון לציון': 'ראשון',
        'Tzur Natan': 'צור נתן',
        "נתניה": "נתנייה"
    }
    df['City'] = df['City'].replace(replace_dict2, regex=True)
    
    # החלפת פסיקים בריקות כדי להסירם
    df['Km'] = df['Km'].str.replace(',', '')
    
    # החלפת ערכים '0' ל-NaN
    df['Km'] = df['Km'].replace('0', np.nan, regex=True)
    df['Km'] = pd.to_numeric(df['Km'], errors='coerce')
    
    # פונקציה להכפלת ערכים קטנים מ-10000 ב-1000
    def multiply_by_1000(x):
        if pd.notna(x) and x < 10000:
            return x * 1000
        else:
            return x

    df['Km'] = df['Km'].apply(multiply_by_1000)
    
    # יצירת OneHotEncoder והחלתו על העמודות
    def one_hot_encode(df, column):
        encoder = OneHotEncoder(sparse=False)
        encoded = encoder.fit_transform(df[[column]])
        encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out([column]))
        df = pd.concat([df, encoded_df], axis=1)
        df.drop(columns=[column], inplace=True)
        return df

    columns_to_encode = ['manufactor', 'model', 'Gear', 'Engine_type', 'Prev_ownership', 'Curr_ownership', 'City', 'Color']
    for column in columns_to_encode:
        df = one_hot_encode(df, column)
    
    # פונקציה לשינוי הערכים בעמודת DESCRIPTION
    def update_description(description):
        has_shamur = 'שמור' in description or 'שמורה' in description
        has_chadash = 'חדש' in description or 'חדשה' in description
        has_matzav_metzuian = 'מצב מצויין' in description or 'מצב מעולה' in description or 'מצב מצוין' in description
        has_matzav_tov = 'מצב טוב' in description
        
        if has_shamur and has_chadash:
            return 'שמור וחדש'
        elif has_shamur:
            return 'שמור'
        elif has_chadash:
            return 'חדש'
        elif has_matzav_metzuian:
            return 'מצב מצויין'
        elif has_matzav_tov:
            return 'מצב טוב'
        else:
            return np.nan

    df['Description'] = df['Description'].apply(update_description)
    df = one_hot_encode(df, 'Description')
    

    
    return df



In [93]:
# קביעת נתיב הקובץ
file_path = r"C:\Users\lior vauonuo\Models\dataset.csv"

# קריאת הקובץ ל-DataFrame
df = pd.read_csv(file_path)


In [94]:
df = prepare_data(df)
df

C:\Users\lior vauonuo\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\lior vauonuo\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\lior vauonuo\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\lior vauonuo\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 

,Year,Hand,capacity_Engine,Price,Pic_num,Km,manufactor_אאודי,manufactor_אופל,manufactor_אלפא רומיאו,manufactor_ב.מ.וו,...,Color_שמפניה,Color_תכלת,Color_תכלת מטאלי,Color_nan,Description_חדש,Description_מצב טוב,Description_מצב מצויין,Description_שמור,Description_שמור וחדש,Description_nan
0,2015,2,1600.0,51000.0,2.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,2018,1,1200.0,49000.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,2010,1,1450.0,22500.0,1.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,2016,1,1600.0,63000.0,5.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2012,1,1248.0,37000.0,1.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,2016,2,1400.0,60000.0,1.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1496,2013,4,1400.0,45000.0,0.0,NaN,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1497,2014,1,1400.0,30000.0,1.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1498,2011,3,1600.0,28000.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [87]:
df_no_nan = df.dropna()
X = df_no_nan.drop('Price', axis=1)
y = df_no_nan['Price']

# נרמול הנתונים
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# פונקציה להערכת ביצועים של מספר שיטות הנדסת פיצ'רים- על פי התוצאות אני בוחר את המודל שבו אשתמש לבחירת הפיצ'רים
def evaluate_model(X, y, model):
    scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')
    mse = -scores.mean()
    
    model.fit(X, y)
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred.round())
    mae = mean_absolute_error(y, y_pred)
    
    return mse, accuracy, mae

# Ridge Regression
ridge = Ridge(alpha=1.0)
mse_ridge, accuracy_ridge, mae_ridge = evaluate_model(X_scaled, y, ridge)

# Forward Selection
def forward_selection(X, y):
    initial_features = []
    best_features = list(X.columns)
    while len(best_features) > 0:
        remaining_features = list(set(X.columns) - set(initial_features))
        new_pval = pd.Series(index=remaining_features)
        for new_column in remaining_features:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[initial_features + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        min_p_value = new_pval.min()
        if min_p_value < 0.05:
            best_feature = new_pval.idxmin()
            initial_features.append(best_feature)
        else:
            break
    return initial_features

selected_features_forward = forward_selection(X, y)
X_forward = X[selected_features_forward]
X_scaled_forward = scaler.fit_transform(X_forward)

logistic = LogisticRegression(max_iter=1000)
mse_forward, accuracy_forward, mae_forward = evaluate_model(X_scaled_forward, y, logistic)

# Backward Elimination
def backward_elimination(X, y):
    features = list(X.columns)
    while len(features) > 0:
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[features]))).fit()
        p_values = model.pvalues[1:]  # ignore the intercept
        max_p_value = p_values.max()
        if max_p_value >= 0.05:
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break
    return features

selected_features_backward = backward_elimination(X, y)
X_backward = X[selected_features_backward]
X_scaled_backward = scaler.fit_transform(X_backward)

mse_backward, accuracy_backward, mae_backward = evaluate_model(X_scaled_backward, y, logistic)

# Correlation-based Feature Selection
correlation_matrix = X.corr().abs()
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool))
selected_features_correlation = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.8)]
X_correlation = X[selected_features_correlation]
X_scaled_correlation = scaler.fit_transform(X_correlation)

mse_correlation, accuracy_correlation, mae_correlation = evaluate_model(X_scaled_correlation, y, logistic)

# הדפסת התוצאות
results = pd.DataFrame({
    'Method': ['Ridge Regression', 'Forward Selection', 'Backward Elimination', 'Correlation-based Selection'],
    'MSE': [mse_ridge, mse_forward, mse_backward, mse_correlation],
    'Accuracy': [accuracy_ridge, accuracy_forward, accuracy_backward, accuracy_correlation],
    'MAE': [mae_ridge, mae_forward, mae_backward, mae_correlation]
})

print(results)


C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' ins

C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' ins

C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\1342082310.py:31: FutureWarning: The default dtype for empty Series will be 'object' ins

                        Method           MSE  Accuracy          MAE
0             Ridge Regression  2.721979e+08  0.000000   163.063547
1            Forward Selection  3.366268e+08  0.925926   111.111111
2         Backward Elimination  3.847946e+08  1.000000     0.000000
3  Correlation-based Selection  5.183367e+08  0.814815  2490.740741


In [96]:


# פונקציית forward_selection
def forward_selection(X, y, significance_level=0.05):
    initial_features = []
    remaining_features = list(X.columns)
    
    while remaining_features:
        new_pval = pd.Series(index=remaining_features)
        
        for new_column in remaining_features:
            model = sm.OLS(y, sm.add_constant(X[initial_features + [new_column]])).fit()
            new_pval[new_column] = model.pvalues[new_column]
        
        min_p_value = new_pval.min()
        
        if min_p_value < significance_level:
            best_feature = new_pval.idxmin()
            initial_features.append(best_feature)
            remaining_features.remove(best_feature)
        else:
            break
    
    return initial_features

X = df.drop(columns=['Price'])  # התאמת שם עמודת היעד
y = df['Price']

# בודק אם יש ערכים חסרים ומסיר אותם
X = X.dropna()
y = y.loc[X.index]

# שימוש בפונקציה לבחירת פיצ'רים
selected_features = forward_selection(X, y)
print("Selected features:", selected_features)

# שימוש בפיצ'רים שנבחרו
X_selected = X[selected_features]

# סטנדרטיזציה של הנתונים
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

# בניית מודל Elastic Net עם Cross-Validation
elastic_net_cv = ElasticNetCV(cv=10, random_state=42)
elastic_net_cv.fit(X_scaled, y)

# חיזוי באמצעות 10-fold cross-validation
y_pred_cv = cross_val_predict(elastic_net_cv, X_scaled, y, cv=10)

# הערכת ביצועי המודל
mse_cv = mean_squared_error(y, y_pred_cv)
rmse_cv = np.sqrt(mse_cv)
r2_cv = r2_score(y, y_pred_cv)
print("Root Mean Squared Error (RMSE) with 10-fold CV:", rmse_cv)
print("R^2 Score with 10-fold CV:", r2_cv)

# מציאת 5 המאפיינים עם ההשפעה הגדולה ביותר
coefficients = elastic_net_cv.coef_
coef_abs = np.abs(coefficients)
top_5_features = np.argsort(coef_abs)[-5:]

print("Top 5 features with the greatest impact:")
for idx in top_5_features:
    feature_name = selected_features[idx]
    feature_impact = coefficients[idx]
    impact_type = "Positive" if feature_impact > 0 else "Negative"
    print(f"Feature: {feature_name}, Impact: {feature_impact}, Type: {impact_type}")


C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead

C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead

C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=remaining_features)
C:\Users\lior vauonuo\AppData\Local\Temp\ipykernel_12728\3918363706.py:7: FutureWarning: The default dtype for empty Series will be 'object' instead

Selected features: ['Year', 'City_חיפה', 'Gear_אוטומטית', 'model_אוקטביה RS', 'model_סראטו', 'City_טייבה', 'model_אלנטרה', 'manufactor_מרצדס', 'manufactor_טויוטה', 'City_שפרעם', 'model_E-Class', 'Prev_ownership_חברה', 'model_אוונסיס', 'Prev_ownership_ליסינג', 'City_באר שבע', 'manufactor_פורד', 'model_פוקוס', 'manufactor_סקודה', 'model_קרוסאובר', 'capacity_Engine', 'City_רעננה', 'model_SX4 קרוסאובר', 'manufactor_אאודי', 'manufactor_קרייזלר', 'City_קרית', 'model_A4', 'Description_מצב טוב', "model_גראנד, וויאג'ר", 'City_קרית ביאליק', 'City_חולון', 'Curr_ownership_חברה', 'manufactor_ניסאן', 'model_מיקרה', 'Color_כסוף', 'model_i10', 'manufactor_רנו', 'manufactor_סוזוקי', 'model_קפצ`ור', 'City_ערערה', 'City_נהריה', 'model_אוקטביה', 'Color_None', 'model_פאסאט CC', 'Color_חום', 'model_i20', 'manufactor_יונדאי', 'model_אסטרה', 'model_קורסה', 'model_סלריו', 'City_דאלית אל כרמל', 'City_נתנייה', 'Prev_ownership_nan', 'Curr_ownership_nan', 'manufactor_מאזדה', 'model_3', 'City_עין מאהל', 'manufactor